<a href="https://colab.research.google.com/github/olga-terekhova/tdsb-calendar/blob/main/TDSB_Calendar_Elementary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create calendars based on the Day 1-5 cycle in TDSB

This notebook takes parameters of the current school year (dates of start and end of the school year, dates of all non-school dates, daily activity schedule) and generates iCal (ICS) calendars that can be imported into Google Calendar.


Calendars with one full-day event per day:
* calendar_school_days.ics - all regular school weekdays labeled with their numbers from 1 to 5
* calendar_no_school_days.ics - all non-school weekdays (PA days, holidays, break boards) labeled with the reason why there is no school
* calendar_all.ics - all weekdays, regular school and non-school  

Calendar with timed intra-day activity events:
* calendar_schedule.ics - a daily schedule for each school weekday broken down into activities

## Provide input parameters  
- First and last day
- PA Days
- Holidays
- Board Breaks
- Daily schedule for Days 1-5

In [1]:
# Define the start and end dates

start_date = '2025-09-02'
end_date = '2026-06-26'

In [2]:
# Define a set of PA days

pa_dates = ['2025-09-26', '2025-10-10', '2025-11-14', '2026-01-16', '2026-02-13', '2026-06-05', '2026-06-26']


In [3]:
# Define a set of holiday days
# Dates are preceded by a label with the holiday name, separated by ": "

holiday_dates = [
    'Labour Day:2025-09-01',
    'Thanksgiving:2025-10-13',
    'Family Day:2026-02-16',
    'Good Friday:2026-04-03',
    'Easter Monday:2026-04-06',
    'Victoria Day:2026-05-18'
]



In [4]:
# Define a list of board breaks
# Each break preceded by a label with the break name, separated by ": "
# Each break represented by a date range, start date and end date separated by ": "

break_dates = [
    'Winter Break: 2025-12-22: 2026-01-02',
    'Mid-Winter Break: 2026-03-16: 2026-03-20'
]



In [5]:
# Define a scope for the daily schedule

start_schedule = '2025-09-02'
end_schedule = '2025-09-30'

In [6]:
# Define the room schedule as a list of lists

schedule_data = [
    ['Day 1',  '08:55:00', '09:00:00', 'Attendance'],
    ['Day 1',  '09:00:00', '09:20:00', 'Entry, Circle Time, Calendar'],
    ['Day 1',  '11:30:00', '12:00:00', 'Lunch'],
    ['Day 1',  '14:45:00', '15:15:00', 'Outdoor Play / Dismissal 3:10 Ready for Dismissal'],
    ['Day 2',  '08:55:00', '09:00:00', 'Attendance'],
    ['Day 2',  '09:00:00', '09:20:00', 'Entry, Circle Time, Calendar'],
    ['Day 2',  '11:30:00', '12:00:00', 'Lunch'],
    ['Day 2',  '14:45:00', '15:15:00', 'Outdoor Play / Dismissal 3:10 Ready for Dismissal'],
    ['Day 3',  '08:55:00', '09:00:00', 'Attendance'],
    ['Day 3',  '09:00:00', '09:20:00', 'Entry, Circle Time, Calendar'],
    ['Day 3',  '11:30:00', '12:00:00', 'Lunch'],
    ['Day 3',  '14:45:00', '15:15:00', 'Outdoor Play / Dismissal 3:10 Ready for Dismissal'],
    ['Day 4',  '08:55:00', '09:00:00', 'Attendance'],
    ['Day 4',  '09:00:00', '09:20:00', 'Entry, Circle Time, Calendar'],
    ['Day 4',  '11:30:00', '12:00:00', 'Lunch'],
    ['Day 4',  '14:45:00', '15:15:00', 'Outdoor Play / Dismissal 3:10 Ready for Dismissal'],
    ['Day 5',  '08:55:00', '09:00:00', 'Attendance'],
    ['Day 5',  '09:00:00', '09:20:00', 'Entry, Circle Time, Calendar'],
    ['Day 5',  '11:30:00', '12:00:00', 'Lunch'],
    ['Day 5',  '14:45:00', '15:15:00', 'Outdoor Play / Dismissal 3:10 Ready for Dismissal']
    ]

## See description of output attributes

**Date**: Calendar date between start and end date of the school year.
* ISO date format.
* Example: '2023-10-25'

**School_Day**: Indicates if there is school on this day.
* Boolean
* True if there is school on Date, False if there is no school on Date

**Type_of_Day**: Type of a day for Date.
* List of values: 'School Day', 'Weekend', 'PA Day', 'Holiday', 'Board Break'
* For days with Type_of_Day = 'School Day', attribute School_Day should be True, and for all other Type_of_Day values School_Day should be False
* If a weekend day is within the range of a board break, the value should be 'Weekend', not 'Board Break'

**Label**: More information about the non-school weekday
* Filled with the name of the holiday for Type_of_Day = 'Holiday'
* Filled with the name of the board break for 'Type_of_Day' = 'Board Break'
* Filled with 'PA Day' for PA Days
* Not filled for regular school days
* Not filled for weekends
* Example: 'Thanksgiving', 'Winter Break'

**Day_of_Week**: Day of the week, from Monday to Sunday.
* Full name (Monday, not Mon)

**Day_Number**: A number from 1 to 5 according to the TDSB day cycle.
* The calendar starts with 1 on the first school day
* Day number increments for each subsequent school day (1, 2, 3, 4, 5), it start from 1 after reaching 5, and it skips all non-school days (weekends, holidays, PA days, board breaks).
* Filled with values from 1 to 5 for all regular school days ('School_Day' = True)
* Not filled for all other days

**Overall_Day_Count**: The overall number of the school day from first day to last day.
* Starts with 1 on the first school day
* Increments for each subsequent school day until the last school day, counting how many school days there are in the current school year
* Filled with values 1+ for all regular school days ('School_Day' = True)
* Not filled for all other days

**Calendar_Display**: A string for representing the day in the resulting calendar.
* For school days ('Type_of_Day' = 'School Day') is filled with 'Day '+ Day_Number. Possible values: 'Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'
* For non-school weekdays ('Type_of_Day' in ['PA Day', 'Holiday', 'Board Break'] is filled with the value of Label
* For weekends ('Type_of_Day' = 'Weekend') is not filled.

**Day_Number**: Day number 1 to 5.
* A string filled with  'Day '+ Day_Number. Possible values: 'Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'

**Begin_Time**: Time when an activity begins.
* Local time (America/Toronto)

**Begin_Time_UTC**: Time when an activity begins in UTC.
* UTC time

**End_Time**: Time when an activity ends.
* Local time (America/Toronto)

**End_Time_UTC**: Time when an activity ends in UTC.
* UTC time

**Activity**: A string describing class activity / lesson.

## Run code

### Import libraries

In [7]:
# Install ics library to work with iCalendar format
!pip install ics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 2.3 MB/s eta 0:00:00


In [8]:
# Import Calendar, Event to work with calendar events in iCalendar
from ics import Calendar, Event

In [9]:
# Import pandas
import pandas as pd

### Create Day 1-5 calendars for the current year

#### Create a dataset with dates

In [10]:
# Create a main series of days between the start and end of school

# Create the date range
date_series = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a DataFrame with "Date" as the column name
df_year_dates = pd.DataFrame({'Date': date_series})

# Display the DataFrame
print(df_year_dates)

          Date
0   2025-09-02
1   2025-09-03
2   2025-09-04
3   2025-09-05
4   2025-09-06
..         ...
293 2026-06-22
294 2026-06-23
295 2026-06-24
296 2026-06-25
297 2026-06-26

[298 rows x 1 columns]


In [11]:
# Create a series of PA days

# Create a range of PA days dates
date_series_pa = pd.to_datetime(pa_dates)

# Create a dataframe with School Day and Type Of Day filled
df_pa_days = pd.DataFrame({'Date': date_series_pa, 'School_Day': False, 'Type_of_Day': 'PA Day', 'Label': 'PA Day'})
print (df_pa_days)

        Date  School_Day Type_of_Day   Label
0 2025-09-26       False      PA Day  PA Day
1 2025-10-10       False      PA Day  PA Day
2 2025-11-14       False      PA Day  PA Day
3 2026-01-16       False      PA Day  PA Day
4 2026-02-13       False      PA Day  PA Day
5 2026-06-05       False      PA Day  PA Day
6 2026-06-26       False      PA Day  PA Day


In [12]:
# Create a DataFrame for Holidays

df_holiday_dates = pd.DataFrame([line.split(':') for line in holiday_dates], columns=['Label', 'Date'])
df_holiday_dates['Date'] = pd.to_datetime(df_holiday_dates['Date'])
df_holiday_dates['School_Day'] = False
df_holiday_dates['Type_of_Day'] = 'Holiday'
df_holiday_dates = df_holiday_dates[['Date', 'School_Day', 'Type_of_Day', 'Label']]

print(df_holiday_dates)


        Date  School_Day Type_of_Day          Label
0 2025-09-01       False     Holiday     Labour Day
1 2025-10-13       False     Holiday   Thanksgiving
2 2026-02-16       False     Holiday     Family Day
3 2026-04-03       False     Holiday    Good Friday
4 2026-04-06       False     Holiday  Easter Monday
5 2026-05-18       False     Holiday   Victoria Day


In [13]:
# Create a series of board breaks

# Initialize an empty list to store DataFrames
break_dataframes = []

# Process each item in the break_dates array
for date_entry in break_dates:
    label, start_date, end_date = date_entry.split(': ')

    # Create a date range between the start and end dates of the current break
    date_range = pd.date_range(start=start_date, end=end_date)

    # Create a DataFrame for the date range
    df = pd.DataFrame({'Date': pd.to_datetime(date_range), 'Label': label})

    # Append the DataFrame to the list
    break_dataframes.append(df)

# Concatenate (union) all DataFrames into one
final_break_df = pd.concat(break_dataframes, ignore_index=True)


final_break_df ['School_Day'] = False
final_break_df ['Type_of_Day'] = 'Board Break'

df_break_dates = final_break_df[['Date', 'School_Day', 'Type_of_Day', 'Label']]
# Display the final DataFrame
print(final_break_df)

         Date             Label  School_Day  Type_of_Day
0  2025-12-22      Winter Break       False  Board Break
1  2025-12-23      Winter Break       False  Board Break
2  2025-12-24      Winter Break       False  Board Break
3  2025-12-25      Winter Break       False  Board Break
4  2025-12-26      Winter Break       False  Board Break
5  2025-12-27      Winter Break       False  Board Break
6  2025-12-28      Winter Break       False  Board Break
7  2025-12-29      Winter Break       False  Board Break
8  2025-12-30      Winter Break       False  Board Break
9  2025-12-31      Winter Break       False  Board Break
10 2026-01-01      Winter Break       False  Board Break
11 2026-01-02      Winter Break       False  Board Break
12 2026-03-16  Mid-Winter Break       False  Board Break
13 2026-03-17  Mid-Winter Break       False  Board Break
14 2026-03-18  Mid-Winter Break       False  Board Break
15 2026-03-19  Mid-Winter Break       False  Board Break
16 2026-03-20  Mid-Winter Break

In [14]:
# Create the final dataframe with all no school days

df_noschool_dates = pd.concat([df_pa_days, df_holiday_dates, df_break_dates], ignore_index = True)
# df_noschool_dates = pd.concat([df_pa_days, df_holiday_dates], ignore_index = True)

print(df_noschool_dates)

         Date  School_Day  Type_of_Day             Label
0  2025-09-26       False       PA Day            PA Day
1  2025-10-10       False       PA Day            PA Day
2  2025-11-14       False       PA Day            PA Day
3  2026-01-16       False       PA Day            PA Day
4  2026-02-13       False       PA Day            PA Day
5  2026-06-05       False       PA Day            PA Day
6  2026-06-26       False       PA Day            PA Day
7  2025-09-01       False      Holiday        Labour Day
8  2025-10-13       False      Holiday      Thanksgiving
9  2026-02-16       False      Holiday        Family Day
10 2026-04-03       False      Holiday       Good Friday
11 2026-04-06       False      Holiday     Easter Monday
12 2026-05-18       False      Holiday      Victoria Day
13 2025-12-22       False  Board Break      Winter Break
14 2025-12-23       False  Board Break      Winter Break
15 2025-12-24       False  Board Break      Winter Break
16 2025-12-25       False  Boar

In [15]:
# Create a dataset with all dates and information from the no school ranges

df_all_dates = pd.merge(df_year_dates, df_noschool_dates, on='Date', how='left', suffixes=('_left', '_right'))

# Add day of week
df_all_dates['Day_of_Week'] = df_all_dates['Date'].dt.day_name()



In [16]:
# peek inside

print(df_all_dates[df_all_dates['Date']=='2025-06-30'])

Empty DataFrame
Columns: [Date, School_Day, Type_of_Day, Label, Day_of_Week]
Index: []


In [17]:
# Fill in regular school days and weekends

# Update rows where Day_of_Week is Saturday or Sunday
df_all_dates.loc[(df_all_dates['Day_of_Week'] == 'Saturday') | (df_all_dates['Day_of_Week'] == 'Sunday'), 'School_Day'] = False
df_all_dates.loc[(df_all_dates['Day_of_Week'] == 'Saturday') | (df_all_dates['Day_of_Week'] == 'Sunday'), 'Type_of_Day'] = 'Weekend'

# Update rows where School_Day is not equal to False
df_all_dates.loc[df_all_dates['School_Day'] != False, 'School_Day'] = True
df_all_dates.loc[df_all_dates['School_Day'] == True, 'Type_of_Day'] = 'School Day'

print(df_all_dates.head(50))

         Date School_Day Type_of_Day         Label Day_of_Week
0  2025-09-02       True  School Day           NaN     Tuesday
1  2025-09-03       True  School Day           NaN   Wednesday
2  2025-09-04       True  School Day           NaN    Thursday
3  2025-09-05       True  School Day           NaN      Friday
4  2025-09-06      False     Weekend           NaN    Saturday
5  2025-09-07      False     Weekend           NaN      Sunday
6  2025-09-08       True  School Day           NaN      Monday
7  2025-09-09       True  School Day           NaN     Tuesday
8  2025-09-10       True  School Day           NaN   Wednesday
9  2025-09-11       True  School Day           NaN    Thursday
10 2025-09-12       True  School Day           NaN      Friday
11 2025-09-13      False     Weekend           NaN    Saturday
12 2025-09-14      False     Weekend           NaN      Sunday
13 2025-09-15       True  School Day           NaN      Monday
14 2025-09-16       True  School Day           NaN     

In [18]:
# Assign day numbers for school days

# Sort by Date first
df_all_dates = df_all_dates.sort_values(by='Date', ascending=True)

# Initialize variables for day number and day count
day_number = 0
day_count = 0
all_day_count = 0

# Create a list to store the Day_Number values
day_numbers = []
all_day_numbers = []

# Iterate through the rows of the DataFrame
for _, row in df_all_dates.iterrows():
    if row['School_Day'] == True:
        all_day_count += 1
        all_day_number = all_day_count
        day_count += 1
        if day_count > 5:
          day_count = 1
        day_number = day_count
    else:
        day_number = None
        all_day_number = None
    day_numbers.append(day_number)
    all_day_numbers.append(all_day_number)

# Add the Day_Number column to the DataFrame
df_all_dates['Day_Number'] = day_numbers
df_all_dates['Overall_Day_Count'] = all_day_numbers

# Display the updated DataFrame
print(df_all_dates.head(100))

         Date School_Day Type_of_Day Label Day_of_Week  Day_Number  \
0  2025-09-02       True  School Day   NaN     Tuesday         1.0   
1  2025-09-03       True  School Day   NaN   Wednesday         2.0   
2  2025-09-04       True  School Day   NaN    Thursday         3.0   
3  2025-09-05       True  School Day   NaN      Friday         4.0   
4  2025-09-06      False     Weekend   NaN    Saturday         NaN   
..        ...        ...         ...   ...         ...         ...   
95 2025-12-06      False     Weekend   NaN    Saturday         NaN   
96 2025-12-07      False     Weekend   NaN      Sunday         NaN   
97 2025-12-08       True  School Day   NaN      Monday         1.0   
98 2025-12-09       True  School Day   NaN     Tuesday         2.0   
99 2025-12-10       True  School Day   NaN   Wednesday         3.0   

    Overall_Day_Count  
0                 1.0  
1                 2.0  
2                 3.0  
3                 4.0  
4                 NaN  
..             

In [19]:
print(df_all_dates.head(301))

          Date School_Day Type_of_Day   Label Day_of_Week  Day_Number  \
0   2025-09-02       True  School Day     NaN     Tuesday         1.0   
1   2025-09-03       True  School Day     NaN   Wednesday         2.0   
2   2025-09-04       True  School Day     NaN    Thursday         3.0   
3   2025-09-05       True  School Day     NaN      Friday         4.0   
4   2025-09-06      False     Weekend     NaN    Saturday         NaN   
..         ...        ...         ...     ...         ...         ...   
293 2026-06-22       True  School Day     NaN      Monday         4.0   
294 2026-06-23       True  School Day     NaN     Tuesday         5.0   
295 2026-06-24       True  School Day     NaN   Wednesday         1.0   
296 2026-06-25       True  School Day     NaN    Thursday         2.0   
297 2026-06-26      False      PA Day  PA Day      Friday         NaN   

     Overall_Day_Count  
0                  1.0  
1                  2.0  
2                  3.0  
3                  4.0 

In [20]:
# Define function to fill value for Calendar_Display

def create_calendar_display(row):
    if row['Type_of_Day'] == 'School Day':
        return "Day " + str(int(row['Day_Number']))
    elif (row['Type_of_Day'] != 'No school day') & (row['Type_of_Day']!='Weekend'):
        return row['Label']
    else:
        return None


# Add calendar display row to manage how an entry looks in a calendar
df_all_dates['Calendar_Display'] = df_all_dates.apply(create_calendar_display, axis=1)

print(df_all_dates.head(50))

         Date School_Day Type_of_Day         Label Day_of_Week  Day_Number  \
0  2025-09-02       True  School Day           NaN     Tuesday         1.0   
1  2025-09-03       True  School Day           NaN   Wednesday         2.0   
2  2025-09-04       True  School Day           NaN    Thursday         3.0   
3  2025-09-05       True  School Day           NaN      Friday         4.0   
4  2025-09-06      False     Weekend           NaN    Saturday         NaN   
5  2025-09-07      False     Weekend           NaN      Sunday         NaN   
6  2025-09-08       True  School Day           NaN      Monday         5.0   
7  2025-09-09       True  School Day           NaN     Tuesday         1.0   
8  2025-09-10       True  School Day           NaN   Wednesday         2.0   
9  2025-09-11       True  School Day           NaN    Thursday         3.0   
10 2025-09-12       True  School Day           NaN      Friday         4.0   
11 2025-09-13      False     Weekend           NaN    Saturday  

#### Create iCalendar export files

In [21]:
# Create iCalendar objects
cal_school = Calendar() # a calendar for all school weekdays (only regular school days)
cal_no_school = Calendar() # a calendar for all non-school weekdays (only PA days, holidays, breaks)
cal_all = Calendar() # a calendar for all weekdays (both school and non-school days)

# Iterate through the DataFrame to create events
for _, row in df_all_dates.iterrows():
    if (row['Calendar_Display'] is not None) :
        event = Event()
        event.name = row['Calendar_Display']
        event.begin = row['Date']  # Set the event date
        event.end = row['Date'] # Set the end date
        event.make_all_day()  # Make the event an all-day event

        # Add the event to one of two calendars
        if (row['School_Day']==True):
          cal_school.events.add(event)
        else:
          cal_no_school.events.add(event)

        cal_all.events.add(event)


# Save the iCalendar data to files
with open('calendar_school_days.ics', 'w') as f:
    f.writelines(cal_school.serialize_iter())

with open('calendar_no_school_days.ics', 'w') as f:
    f.writelines(cal_no_school.serialize_iter())

with open('calendar_all.ics', 'w') as f:
    f.writelines(cal_all.serialize_iter())

print("iCalendar file created: calendar_school_days.ics, calendar_no_school_days.ics, calendar_all.ics")

iCalendar file created: calendar_school_days.ics, calendar_no_school_days.ics, calendar_all.ics


### Create a daily schedule based on the Day 1-5 calendar

#### Create a dataset with dates, times and activities

In [22]:
# Create a DataFrame out of schedule data
df_day_schedule = pd.DataFrame (schedule_data, columns = ['Day_Number', 'Begin_Time', 'End_Time', 'Activity'])

print(df_day_schedule)

   Day_Number Begin_Time  End_Time  \
0       Day 1   08:55:00  09:00:00   
1       Day 1   09:00:00  09:20:00   
2       Day 1   11:30:00  12:00:00   
3       Day 1   14:45:00  15:15:00   
4       Day 2   08:55:00  09:00:00   
5       Day 2   09:00:00  09:20:00   
6       Day 2   11:30:00  12:00:00   
7       Day 2   14:45:00  15:15:00   
8       Day 3   08:55:00  09:00:00   
9       Day 3   09:00:00  09:20:00   
10      Day 3   11:30:00  12:00:00   
11      Day 3   14:45:00  15:15:00   
12      Day 4   08:55:00  09:00:00   
13      Day 4   09:00:00  09:20:00   
14      Day 4   11:30:00  12:00:00   
15      Day 4   14:45:00  15:15:00   
16      Day 5   08:55:00  09:00:00   
17      Day 5   09:00:00  09:20:00   
18      Day 5   11:30:00  12:00:00   
19      Day 5   14:45:00  15:15:00   

                                             Activity  
0                                          Attendance  
1                        Entry, Circle Time, Calendar  
2                                

In [23]:
# Prepare school year schedule and daily schedule for join using 'Calendar_Display' = 'Day_Number'
df_day_schedule['Calendar_Display'] = df_day_schedule['Day_Number']

# Keep only school weekdays in the daily schedule scope (see Input Parameters)
start_schedule_date = pd.to_datetime(start_schedule)
end_schedule_date = pd.to_datetime(end_schedule)

df_schedule_dates = df_all_dates[(df_all_dates['Date']<= end_schedule_date) &
 (df_all_dates['Date']>=start_schedule_date) & (df_all_dates['School_Day']==True)]

# Left join school year schedule and daily schedule
df_all_schedule = pd.merge(df_schedule_dates, df_day_schedule, on='Calendar_Display', how='left', suffixes=('_left', '_right'))
df_all_schedule = df_all_schedule[['Date', 'Calendar_Display', 'Begin_Time', 'End_Time', 'Activity']]

# Make a date time out of a Date and Time for begin and end of activities
df_all_schedule['Begin_Time'] = df_all_schedule ['Date'].dt.strftime('%Y-%m-%d') + ' ' + df_all_schedule['Begin_Time']
df_all_schedule['Begin_Time'] = pd.to_datetime(df_all_schedule['Begin_Time'])

df_all_schedule['End_Time'] = df_all_schedule ['Date'].dt.strftime('%Y-%m-%d') + ' ' + df_all_schedule['End_Time']
df_all_schedule['End_Time'] = pd.to_datetime(df_all_schedule['End_Time'])

print(df_all_schedule)

         Date Calendar_Display          Begin_Time            End_Time  \
0  2025-09-02            Day 1 2025-09-02 08:55:00 2025-09-02 09:00:00   
1  2025-09-02            Day 1 2025-09-02 09:00:00 2025-09-02 09:20:00   
2  2025-09-02            Day 1 2025-09-02 11:30:00 2025-09-02 12:00:00   
3  2025-09-02            Day 1 2025-09-02 14:45:00 2025-09-02 15:15:00   
4  2025-09-03            Day 2 2025-09-03 08:55:00 2025-09-03 09:00:00   
..        ...              ...                 ...                 ...   
75 2025-09-29            Day 4 2025-09-29 14:45:00 2025-09-29 15:15:00   
76 2025-09-30            Day 5 2025-09-30 08:55:00 2025-09-30 09:00:00   
77 2025-09-30            Day 5 2025-09-30 09:00:00 2025-09-30 09:20:00   
78 2025-09-30            Day 5 2025-09-30 11:30:00 2025-09-30 12:00:00   
79 2025-09-30            Day 5 2025-09-30 14:45:00 2025-09-30 15:15:00   

                                             Activity  
0                                          Attendance  

In [24]:
# Working with timezones

# Make the dataset timezone-aware
df_all_schedule['Begin_Time'] = df_all_schedule['Begin_Time'].dt.tz_localize('America/Toronto')
df_all_schedule['End_Time'] = df_all_schedule['End_Time'].dt.tz_localize('America/Toronto')

# Covert datetimes into UTC for the iCalendar format
df_all_schedule['Begin_Time_UTC'] = df_all_schedule['Begin_Time'].dt.tz_convert('UTC')
df_all_schedule['End_Time_UTC'] = df_all_schedule['End_Time'].dt.tz_convert('UTC')

In [25]:
df_print_month = df_all_schedule[(df_all_schedule['Date']>'2023-10-01')&(df_all_schedule['Date']<'2023-10-31')]
df_print_month = df_print_month[['Begin_Time', 'Begin_Time_UTC']]
print(df_print_month)

Empty DataFrame
Columns: [Begin_Time, Begin_Time_UTC]
Index: []


#### Create an iCalendar export file

In [26]:
# Create an iCalendar object
cal_schedule = Calendar()

# Iterate through the DataFrame to create events
for _, row in df_all_schedule.iterrows():
    if (row['Calendar_Display'] is not None) :
        event = Event()
        event.name = row['Activity']
        event.begin = row['Begin_Time_UTC']  # Set the event date
        event.end = row['End_Time_UTC'] # Set the end date
        cal_schedule.events.add(event)


# Save the iCalendar data to files
with open('calendar_schedule.ics', 'w') as f:
    f.writelines(cal_schedule.serialize_iter())

print("iCalendar file created: calendar_schedule.ics")

iCalendar file created: calendar_schedule.ics
